In [4]:
import pandas as pd
import numpy as np
import duckdb

con = duckdb.connect('/home/rgc/projects/dlt/Homework/taxi_pipeline.duckdb', read_only=True)


In [5]:
con.sql("DESCRIBE")

┌───────────────┬──────────────────────┬─────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────┐
│   database    │        schema        │        name         │                                                                                                                   column_names                                                                                                                   │                                                                                       column_types                                                                                       │ te

## What is the start date and end date of the dataset?

In [6]:
con.sql("SELECT MIN(trip_pickup_date_time), MAX(trip_pickup_date_time) FROM taxi_dataset.taxi_zones")

┌────────────────────────────┬────────────────────────────┐
│ min(trip_pickup_date_time) │ max(trip_pickup_date_time) │
│  timestamp with time zone  │  timestamp with time zone  │
├────────────────────────────┼────────────────────────────┤
│ 2009-06-01 13:43:00+02     │ 2009-06-30 23:05:00+02     │
└────────────────────────────┴────────────────────────────┘

## What proportion of trips are paid with credit card?

In [7]:
# Vérifier les types de paiement disponibles
con.sql("SELECT DISTINCT payment_type FROM taxi_dataset.taxi_zones ORDER BY payment_type")

┌──────────────┐
│ payment_type │
│   varchar    │
├──────────────┤
│ CASH         │
│ Cash         │
│ Credit       │
└──────────────┘

In [10]:
# Calculer la proportion de trajets payés avec une carte de crédit
con.sql("""
WITH trip_stats AS (
    SELECT 
        COUNT(*) as total_trips,
        SUM(CASE WHEN payment_type = 'Credit' THEN 1 ELSE 0 END) as credit_trips
    FROM taxi_dataset.taxi_zones
)
SELECT 
    total_trips,
    credit_trips,
    ROUND(100.0 * credit_trips / total_trips, 2) as proportion_percent
FROM trip_stats
""")

┌─────────────┬──────────────┬────────────────────┐
│ total_trips │ credit_trips │ proportion_percent │
│    int64    │    int128    │       double       │
├─────────────┼──────────────┼────────────────────┤
│        1000 │          257 │               25.7 │
└─────────────┴──────────────┴────────────────────┘